In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

def generate_ground_truth_keypoints(video_dir, output_dir="ground_truth"):
    """Generate keypoint annotations using MoveNet."""
    # Load MoveNet
    print("Loading MoveNet model...")
    model = hub.load('https://tfhub.dev/google/movenet/singlepose/lightning/4')
    movenet = model.signatures['serving_default']
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Process each video
    for video_file in os.listdir(video_dir):
        if not video_file.endswith(('.mp4', '.avi', '.mov')):
            continue
            
        video_name = os.path.splitext(video_file)[0]
        video_path = os.path.join(video_dir, video_file)
        output_file = os.path.join(output_dir, f"{video_name}_keypoints.npy")
        
        print(f"\nProcessing {video_file}...")
        
        # Read video
        cap = cv2.VideoCapture(video_path)
        frames = []
        keypoints = []
        frame_count = 0
        
        while True:
            success, frame = cap.read()
            if not success:
                break
                
            # Process every 10th frame
            if frame_count % 10 == 0:
                # Prepare frame for MoveNet
                img = cv2.resize(frame, (192, 192))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = np.expand_dims(img, axis=0)
                img = tf.cast(img, dtype=tf.int32)
                
                # Get keypoints
                outputs = movenet(input=img)
                kpts = outputs['output_0'].numpy()
                
                # Remove extra dimensions and store keypoints
                kpts = np.squeeze(kpts)  # Remove batch dimension
                keypoints.append(kpts)  # shape: (17, 3) - x, y, confidence
                frames.append(frame)
                
            frame_count += 1
        
        cap.release()
        
        if frames:
            # Convert to numpy arrays
            keypoints = np.array(keypoints)  # shape: (num_frames, 17, 3)
            
            # Save keypoints
            np.save(output_file, keypoints)
            print(f"Saved {len(keypoints)} keypoints to {output_file}")
            print(f"Keypoint shape: {keypoints.shape}")
            print(f"Value range: [{keypoints.min():.3f}, {keypoints.max():.3f}]")
            
            # Visualize first frame for verification
            first_frame = frames[0]
            h, w = first_frame.shape[:2]
            first_kpts = keypoints[0]  # shape: (17, 3)
            
            # Draw keypoints on first frame
            viz_frame = first_frame.copy()
            colors = [(0, 255, 0), (0, 0, 255), (255, 0, 0)]  # Different colors for first 3 keypoints
            
            print("\nFirst frame keypoints (first 3):")
            print(first_kpts[:3])
            
            for i in range(3):  # Draw first 3 keypoints
                x, y, conf = first_kpts[i]  # Now should work correctly
                if conf > 0.3:  # Only draw high confidence points
                    x_px = int(x * w)
                    y_px = int(y * h)
                    cv2.circle(viz_frame, (x_px, y_px), 5, colors[i], -1)
                    cv2.putText(viz_frame, f"kp{i}", (x_px + 5, y_px + 5),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[i], 1)
            
            # Save visualization
            viz_path = os.path.join(output_dir, f"{video_name}_viz.jpg")
            cv2.imwrite(viz_path, viz_frame)
            print(f"Saved visualization to {viz_path}")
            
            # Print keypoint statistics for verification
            print("\nKeypoint statistics:")
            print(f"X-coordinates range: [{keypoints[:, :, 0].min():.3f}, {keypoints[:, :, 0].max():.3f}]")
            print(f"Y-coordinates range: [{keypoints[:, :, 1].min():.3f}, {keypoints[:, :, 1].max():.3f}]")
            print(f"Confidence range: [{keypoints[:, :, 2].min():.3f}, {keypoints[:, :, 2].max():.3f}]")

if __name__ == "__main__":
    # Directory containing your video files
    video_dir = r"C:\Users\uppal\Desktop\HumanPoseDetection"
    
    # Generate keypoints
    generate_ground_truth_keypoints(video_dir)
    
    print("\nKeypoint generation complete!")

2025-01-02 21:00:21.819638: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ImportError: cannot import name 'Circle' from 'keras.src.losses.losses' (/opt/anaconda3/lib/python3.12/site-packages/keras/src/losses/losses.py)